In [4]:
My_student_number = 401722136

In [193]:
import numpy as np



In [204]:
# Thanks to https://github.com/rougier/neural-networks/blob/master/art1.py because of good code in this part
class ART:

    def __init__(self, n, m, rho=.5):
        '''
        Create network with specified shape
        Parameters:
        -----------
        n : int
            Size of input
        m : int
            Maximum number of internal units
        rho : float
            Vigilance parameter
        '''
        # Comparison layer
        self.F1 = np.ones(n)
        # Recognition layer
        self.F2 = np.ones(m)
        # Feed-forward weights
        self.Wf = np.random.random((m,n))
        # Feed-back weights
        self.Wb = np.random.random((n,m))
        # Vigilance
        self.rho = rho
        # Number of active units in F2
        self.active = 0


    def learn(self, X):
        ''' Learn X '''

        # Compute F2 output and sort them (I)
        self.F2[...] = np.dot(self.Wf, X)
        I = np.argsort(self.F2[:self.active].ravel())[::-1]
        for i in I:
            # Check if nearest memory is above the vigilance level
            d = (self.Wb[:,i]*X).sum()/X.sum()
            if d >= self.rho:
                # Learn data
                self.Wb[:,i] *= X
                self.Wf[i,:] = self.Wb[:,i]/(0.5+self.Wb[:,i].sum())
                return self.Wb[:,i], i

        # No match found, increase the number of active units
        # and make the newly active unit to learn data
        if self.active < self.F2.size:
            i = self.active
            self.Wb[:,i] *= X
            self.Wf[i,:] = self.Wb[:,i]/(0.5+self.Wb[:,i].sum())
            self.active += 1
            return self.Wb[:,i], i

        return None,None

    def predict(self, X):
        ''' Test X '''
        # Compute F2 output and sort them (I)
        self.F2[...] = np.dot(self.Wf, X)
        I = np.argsort(self.F2[:self.active].ravel())[::-1]

        bigest = -1
        bigest_index = 0
        for i in I:
            # Check if nearest memory is above the vigilance level
            d = (self.Wb[:,i]*X).sum()/X.sum()
            if bigest > d :
                bigest_index= i
                bigest = d


        return self.Wb[:,bigest_index], bigest_index



In [205]:
set_one_train = {"A" : [[0,1,1,0],
                        [1,0,0,1],
                        [1,1,1,1],
                        [1,0,0,1]],

                 "C":[[0,1,1,1],
                      [1,0,0,0],
                      [1,0,0,0],
                      [0,1,1,1]],

                 "J":[[0,1,1,1],
                      [0,0,1,0],
                      [1,0,1,0],
                      [1,1,1,0]],

                 "L":[[1,0,0,0],
                      [1,0,0,0],
                      [1,0,0,0],
                      [1,1,1,1]]}
set_one_test = {"O":[[0,1,1,0],
                     [1,0,0,1],
                     [1,0,0,1],
                     [0,1,1,0]]}

In [206]:
for i in set_one_train:
    print(i)

A
C
J
L


In [207]:
network = ART( 4*4, 10, rho=0.25)
dic_main = {}
# It give to dictionary that value is list and see are they same as each other
def two_dic_is_same(dic_main, dic_helper):
    list_of_key_main = dic_main.keys()
    for item in dic_helper:
        if item not in list_of_key_main:
            return False
        if np.array_equal(dic_main[item],dic_helper[item]):
            return False
    return True

# we use learning until all our weight not change.
while True:
    dic_helper = {}
    for i in set_one_train:
            # Z is matrix weight of class k
            Z, k = network.learn(np.array(set_one_train[i]).ravel())
            dic_helper[k] = Z

    if not two_dic_is_same(dic_main,dic_helper):
        break
    else:
        dic_main = dic_helper
# see class of each of input item
for i in set_one_train:
            Z, k = network.learn(np.array(set_one_train[i]).ravel())
            print(f"{i}-> class {k}")
            print(Z.reshape(4,4))


A-> class 0
[[0.         0.94833274 0.4942051  0.        ]
 [0.28470968 0.         0.         0.        ]
 [0.56867322 0.         0.         0.        ]
 [0.         0.         0.         0.28458536]]
C-> class 0
[[0.         0.94833274 0.4942051  0.        ]
 [0.28470968 0.         0.         0.        ]
 [0.56867322 0.         0.         0.        ]
 [0.         0.         0.         0.28458536]]
J-> class 1
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.77157846 0.         0.         0.        ]
 [0.21880439 0.97117124 0.89173377 0.        ]]
L-> class 1
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.77157846 0.         0.         0.        ]
 [0.21880439 0.97117124 0.89173377 0.        ]]


In [208]:
# see class of input test
for i in set_one_test:
        Z, k = network.predict(np.array(set_one_test[i]).ravel())
        print(f"{i}-> class {k}")
        print(Z.reshape(4,4))


O-> class 0
[[0.         0.94833274 0.4942051  0.        ]
 [0.28470968 0.         0.         0.        ]
 [0.56867322 0.         0.         0.        ]
 [0.         0.         0.         0.28458536]]
